In [5]:
!pip install opencv-python==4.2.0.34 snorkel==0.9.5 scikit-learn==0.21.3

  Using cached numpy-1.21.4-cp38-cp38-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.16.6
    Uninstalling numpy-1.16.6:
      Successfully uninstalled numpy-1.16.6


In [6]:
import glob
import os
import cv2
import numpy as np
from snorkel.labeling import LFApplier
from snorkel.labeling.model import LabelModel
from Picture import Picture
from PixelData import PixelData
from labeling_fcns import lfs, window_size,h,w
from gold_labels import get_gold_labels

ValueError: numpy.ndarray size changed, may indicate binary incompatibility. Expected 88 from C header, got 80 from PyObject

In [ ]:
gt_path='labeling/gt/' # ground truth
dest_path='labeling/masks_generated/' # labelling fnuction output ?
source_path='labeling/source/*' # raw images

In [ ]:

images_sorted=sorted(glob.glob(source_path))
images_splitted=np.array_split(images_sorted, 26)

rows=h
cols=w

for list_idx,list in enumerate(images_splitted):

        df_train_Pictures = []
        L_train = []
        applier = LFApplier(lfs)

        gold=get_gold_labels(list,gt_path)
        for idx,filepath in enumerate(list):
            filename = os.path.basename(filepath)
            image=cv2.imread(filepath)
            #cv2.namedWindow('gray_scale', cv2.WINDOW_NORMAL)
            #cv2.imshow('gray_scale', image)
            #cv2.waitKey(0) # press numpad 0
            image_pixels = []
            print("filtering img")
            filtered_image = cv2.bilateralFilter(image, 9, 75, 75)
            for x in range(0, rows):
                for y in range(0, cols):
                    cutted_image= filtered_image[x:(x + window_size), y:(y + window_size)]
                    output_labels = applier.apply([cutted_image])
                    pixelData=PixelData(output_labels,x,y)
                    image_pixels.append(pixelData)
                    L_train.append(output_labels[0])
            print("img filtered")

            pic=Picture(image_pixels,filepath)
            df_train_Pictures.append(pic)

        L = np.array(L_train)

        print("training data")
        label_model = LabelModel(cardinality=2, verbose=True, device='cpu')
        #label_model = LabelModel(cardinality=2, verbose=True,device='cuda')
        print("fitting data")
        label_model.fit(L_train=L, seed=123,log_freq=1)

        for picture in df_train_Pictures:
            image=np.zeros((rows,cols))
            for idx2, pixel in enumerate(picture.pixels):
                pred=label_model.predict(pixel.labels)
                if pred>0:
                    pred=0
                else:
                    pred=255
                image[pixel.x,pixel.y]=pred
            current_path=dest_path+os.path.basename(picture.filename)
            cv2.imwrite(current_path, image)
